SIMULATION PROJECT

In [ ]:
Mini Project 1
Two-Transmission-Link Queueing System Simulator and Output Instructions

In this mini-project, you will develop a discrete-event driven simulator for a two-queue transmission system described below, and analyze the simulation results for a number of different settings. To make your task easier, you will be provided with a simulation program written by a former  student with his permission. You may refer to this program to understand the overall structure of your program. However, please do not directly copy portions of the program, but instead use it as a guide.


System Description:
•	Packets arrive (avg. interrarrival time: 1/lambda) to a router with two outgoing transmission links. 
•	Packets arrive to the router according to a Poisson Process. Each arriving packet is instantaneously placed on one of the two outgoing transmission link queues.
•	Service times are distributed exponentially at each transmission link with different average service rates mu1 and mu2.
•	An arriving packet joins link i with a fixed probability i independently of the other packets and the system state.

 

Numerical Results and Analysis:
You will use the two-queue system simulator that you develop to generate a set of results according to the following instructions. 

For each simulation setting below, generate at least 6000 arrivals according to a Poisson Process with rate Lambda. Ignore at least the first 1000 packets to leave the system in your performance metric calculations. If you use simulation time as your stopping criterion, adjust your simulation time accordingly. You may need to increase these values further in order to obtain steady-state behavior. For each set of parameters, show the following metrics in a table:
 
1)	the blocking probability (Pb) (at each queue, and in the system) 
2)	the average delay at each queue (and in the system)
3)	the average throughput at each queue and in the system
4)	the average number of packets in each queue and the system. You may use calculations based on your simulations for item number 4.

Simulation settings:
•	Set mu1 = 5 packets/s; mu2 = 5 packets/s. Set the arrival rate (Lambda) to 8 packets/s
•	Set the buffer size at each queue to 20
•	Simulate the system with the following Phi 1 (the probability of an arriving packet to choose queue 1) values:  0.4; 0.5 ; 0.6 
•	Repeat the experiments above with a buffer size of 5 at each queue


ANSWERS
Additional Questions:
1)	How does the system behavior change with Phi? Explain your observations.
Limited processes are accomodated
2)	How does the system behavior change with the buffer size? Explain your observations.
It can accomodate more processes with increase in buffer size upto an optimum
3)	Which setting results with the highest system (total) throughput? Which setting results with the lowest blocking probability? Explain.
Total Throughput
Buffer Size=30
Phi=10

Lowest Blocking Probability=0.1

In [ ]:

import numpy as np
import pandas as pd

class Packet_Simulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure1=float('inf')      #departure time from server 1
        self.t_departure2=float('inf')      #departure time from server 2
        self.dep_sum1=0                     #Sum of service times by router 1
        self.dep_sum2=0                     #Sum of service times by router 2
        self.state_T1=0                     #current state of server1 (binary)
        self.state_T2=0                     #current state of server2 (binary)
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              # who had to wait in line(counter)
        self.num_in_system=0                #current number of packets in the system
        self.num_of_departures1=0           #number of customers served by router 1  
        self.num_of_departures2=0           #number of customers served by router 2 
        self.lost_packets=0               # left without service
        
    def time_adv(self):                                                       #timing routine
        t_next_event=min(self.t_arrival,self.t_departure1,self.t_departure2)  #determine time of next event
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure1 and self.t_arrival<self.t_departure2:
            self.arrival()
        elif self.t_departure1<self.t_arrival and self.t_departure1<self.t_departure2:
            self.teller1()
        else:
            self.teller2()
            
        
    
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        if self.num_in_q == 0:                              #schedule next departure or arrival depending on state of servers
            if self.state_T1==1 and self.state_T2==1:
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival=self.clock+self.gen_int_arr()
                
                
            elif self.state_T1==0 and self.state_T2==0:
                
                if np.random.choice([0,1])==1:
                    self.state_T1=1
                    self.dep1= self.gen_service_time_teller1()
                    self.dep_sum1 += self.dep1
                    self.t_departure1=self.clock + self.dep1
                    self.t_arrival=self.clock+self.gen_int_arr()

                else:
                    self.state_T2=1
                    self.dep2= self.gen_service_time_teller2()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.t_arrival=self.clock+self.gen_int_arr()

                    
            elif self.state_T1==0 and self.state_T2 ==1:    #if server 2 is busy customer goes to server 1
                self.dep1= self.gen_service_time_teller1()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T1=1
            else:                                           #otherwise customer goes to server 2
                self.dep2= self.gen_service_time_teller2()
                self.dep_sum2 += self.dep2
                self.t_departure2=self.clock + self.dep2
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T2=1
        
        elif self.num_in_q < 4 and self.num_in_q >= 1:
            self.num_in_q+=1
            self.number_in_queue+=1                             #if queue length is less than 4 generate next arrival and make customer join queue
            self.t_arrival=self.clock + self.gen_int_arr()
            
        elif self.num_in_q == 4:                             #if queue length is 4 equal prob to leave or stay
            if np.random.choice([0,1])==0: 
                self.num_in_q+=1 
                self.number_in_queue+=1                 
                self.t_arrival=self.clock + self.gen_int_arr()
            else:
                self.lost_customers+=1
                
                
        elif self.num_in_q >= 5:                            #if queue length is more than 5 60% chance of leaving
            if np.random.choice([0,1],p=[0.4,0.6])==0:
                self.t_arrival=self.clock+self.gen_int_arr()
                self.num_in_q+=1 
                self.number_in_queue+=1 
            else:
                self.lost_customers+=1
                
            

    
    def router1(self):              #departure from server 2
        self.num_of_departures1 += 1
        self.num_in_system -= 1 
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_teller1()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0                  
    
    def router2(self):              #departure from server 1
        self.num_of_departures2 += 1
        self.num_in_system -= 1
        if self.num_in_q>0:
            self.dep2= self.gen_service_time_teller2()
            self.dep_sum2 += self.dep2
            self.t_departure2=self.clock + self.dep2
            self.num_in_q-=1
        else:
            self.t_departure2=float('inf')
            self.state_T2=0
            
    
    def gen_int_arr(self):                                             #function to generate arrival times using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
    
    def gen_service_time_router1(self):                                #function to generate service time for 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.2)
    
    def gen_service_time_router2(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.5)


   
s=Packet_Simulation()
df=pd.DataFrame(columns=['Average interarrival time','Average service time router1','Average service time router 2','Utilization router1','Utilization router 2','Packets  to wait in line','Total average wait time','Lost Packets'])


for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum1/s.num_of_departures1,s.dep_sum2/s.num_of_departures2,s.dep_sum1/s.clock,s.dep_sum2/s.clock,s.number_in_queue,s.total_wait_time,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('results.xlsx')    



    

In [ ]:
DUMMY CODE FOR PARAMETER TESTING

In [ ]:
import queue
import random
import numpy as np

k=input('Enter the buffer size:')
k=int(k)+1
prob_1=input('The  probability  of  an  arriving  packet  to choose queue 1) :')
prob_1=float(prob_1)

rate_l=input('arrival rate(lambda):') #rate_l=8
rate_l=float(rate_l)
rate_m1=input('process rate(mu1):') #rate_m1=5
rate_m1=float(rate_m1)
rate_m2=input('process rate(mu2):') #rate_m2=5
rate_m2=float(rate_m2)
ans1_1=[]
ans1_2=[]
ans1_3=[]
ans2_1=[]
ans2_2=[]
ans2_3=[]
ans3_1=[]
ans3_2=[]
ans3_3=[]
ans4_1=[]
ans4_2=[]
for i in range(5):
    q=queue.Queue(k)
    q2=queue.Queue(k)
    
    
    count=0
    count1=0
    q1_processed=0
    count2_1=0
    q2_processed=0
    t_record1=0
    t_record2=0
    t_record2_2=0
    c=0
    pkt_delay1=[]
    pkt_delay2=[]
    flag1=False
    flag2=False
    timecount1=0
    timecount2=0
    
    
    while(count<5000):
        count+=1
        t=random.expovariate(rate_l)
        t_record1+=t
        while(t_record2<=t_record1 and (not q.empty())):
            if(not flag1):
                t_out=random.expovariate(rate_m1)
                t_record2+=t_out
            if(t_record2<t_record1):
                q.get()
                pkt_delay1[q1_processed]=t_record2-pkt_delay1[q1_processed]
                if(q1_processed==500):
                    timecount1=t_record2
                q.task_done()
                q1_processed+=1
                flag1=False
            else:
                flag1=True
                break
        while(t_record2_2<=t_record1 and (not q2.empty())):
            if(not flag2):
                t_out=random.expovariate(rate_m2)
                t_record2_2+=t_out
            if(t_record2_2<t_record1):
                q2.get()
                pkt_delay2[q2_processed]=t_record2_2-pkt_delay2[q2_processed]
                if(q2_processed==500):
                    timecount2=t_record2_2
                q2.task_done()
                q2_processed+=1
                flag2=False
            else:
                flag2=True
                break
        if(q.empty()):
            t_record2=t_record1
        if(q2.empty()):
            t_record2_2=t_record1
        if(random.random()<prob_1):
            count1+=1
            if(not q.full()):
                q.put(count)
                if(count>1000):
                    c+=1
                pkt_delay1=pkt_delay1+[t_record1]
        else:
            count2_1+=1
            if(not q2.full()):
                q2.put(count)
                if(count>1000):
                    c+=1
                pkt_delay2=pkt_delay2+[t_record1]
    ans1_1=ans1_1+[1-(q1_processed+q2_processed)/(count-q.qsize()-q2.qsize())]
    ans1_2=ans1_2+[1-(q1_processed/(count1-q.qsize()))]
    ans1_3=ans1_3+[1-(q2_processed/(count2_1-q2.qsize()))]
    ans2_1=ans2_1+[sum(pkt_delay1[500:-k])/(len(pkt_delay1)-500-k)]
    ans2_2=ans2_2+[sum(pkt_delay2[500:-k])/(len(pkt_delay2)-500-k)]
    ans2_3=(ans2_1[i]*q1_processed+ans2_2[i]*q2_processed)/(q1_processed+q2_processed)
    ans3_1=ans3_1+[(q1_processed+q2_processed)/t_record1]
    ans3_2=ans3_2+[q1_processed/t_record1]
    ans3_3=ans3_3+[q2_processed/t_record1]
    ans4_1=ans4_1+[ans2_1[-1]*rate_l*prob_1*(1-ans1_2[-1])]
    ans4_2=ans4_2+[ans2_2[-1]*rate_l*(1-prob_1)*(1-ans1_3[-1])]

     
print("Total packets:",count, "packets going to q1:",count1,"q1 processed packets",q1_processed,"packets going to q2",count2_1,"q2 processed packets",q2_processed,c)
print("queue 1 blocking prob:",np.mean(ans1_2))
print("queue 2 blocking prob:",np.mean(ans1_3))
print("system blocking prob:", np.mean(ans1_1))
print("average delay for queue1:",np.mean(ans2_1))
print("average delay for queue2:",np.mean(ans2_2))
print("average delay for system:",np.mean(ans2_3))
print("throughput in queue 1:",np.mean(ans3_2))
print("throughput in queue 2:",np.mean(ans3_3))
print("throughput in the system:", np.mean(ans3_1))
print("average number of packets in queue1",np.mean(ans4_1))
print("average number of packets in queue2",np.mean(ans4_2))
print("average number of packets in system",np.mean(ans4_1)+np.mean(ans4_2))

